[![Foo](https://miro.medium.com/max/857/1*AqqArOvacibWqeulyP_-8Q.png)](http://google.com.au/)

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False

import matplotlib.pyplot as plt

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0' 

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

import random

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# 일단 만들기

In [2]:
class FashionCNN(nn.Module) : 
    def __init__(self) : 
        super(FashionCNN,self).__init__()

        self.cnn_block1 = nn.Sequential(
            # 28x28x1 -> 28x28x32 -> 14x14x32
            nn.Conv2d(1, 32, 3, padding = 1), nn.LeakyReLU(0.2), nn.BatchNorm2d(32), nn.Dropout(0.1),
            nn.Conv2d(32, 32, 3, padding = 1), nn.LeakyReLU(0.2), nn.BatchNorm2d(32),
            nn.MaxPool2d(2,2))
        
        self.cnn_block2 = nn.Sequential(
            # 14x14x32 -> 14x14x64 -> 7x7x64
            nn.Conv2d(32, 64, 3, padding = 1), nn.LeakyReLU(0.2),nn.BatchNorm2d(64), nn.Dropout(0.1),
            nn.Conv2d(64,64,3, padding=1), nn.LeakyReLU(0.2),nn.BatchNorm2d(64),
            nn.MaxPool2d(2,2)
            )

        self.cnn_block3 = nn.Sequential(
            # 7x7x64 -> 7x7x128 -> 1x1x128
            nn.Conv2d(64, 128, 3, padding = 1), nn.LeakyReLU(0.2),nn.BatchNorm2d(128), nn.Dropout(0.1),
            nn.Conv2d(128, 128, 3, padding = 1), nn.LeakyReLU(0.2),nn.BatchNorm2d(128),
            nn.MaxPool2d(7)
            )

        self.fc_layer = nn.Sequential(
            # 128 ->  10
            nn.Linear(128, 10)
        )

    
    def forward(self, x) :
        batch_size = x.size(0)
        x = self.cnn_block1(x)
        x = self.cnn_block2(x)
        x = self.cnn_block3(x)
 
        x = x.view(batch_size,-1)
        
        x = self.fc_layer(x)
        return F.softmax(x)

# Data Loader

In [3]:
batch_size = 512

fmnist_train = dset.FashionMNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
fmnist_test = dset.FashionMNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

In [4]:
from sklearn.model_selection import StratifiedKFold
n_split = 5
skf = StratifiedKFold(n_splits = n_split, shuffle = True)

for train_idx, valid_idx in skf.split(np.arange(fmnist_train.__len__()), fmnist_train.targets) : 
    break;

def return_dataloader(train_idx, valid_idx) : 
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_idx)

    train_data = DataLoader(dataset=fmnist_train, batch_size = batch_size, sampler = train_subsampler)
    valid_data = DataLoader(dataset=fmnist_train, batch_size = batch_size, sampler = valid_subsampler)
    

    dataloaders = {}
    dataloaders['train'] = train_data
    dataloaders['valid'] = valid_data

    return dataloaders

# Train_model

In [5]:
def train_model(model, dataloader, optimizer, criterion, num_epoch, early_stop, model_path) : 
    import time

    start_time = time.time()
    early_stop_epoch = 0
    best_val_loss = np.float('inf')

    for epoch in range(num_epoch) : 
        for phase in ['train','valid'] : 
            if phase == 'train' : 
                model.train()
            elif phase == 'valid' : 
                model.eval()
            
            running_loss = 0
            running_corr = 0
            total = 0 


            for x, y in dataloader[phase] : 
                x = x.to(device)
                y = y.to(device)
                optimizer.zero_grad()
                total += x.size(0)


                with torch.set_grad_enabled(phase == 'train') : 
                    output = model(x)
                    loss = criterion(output, y)
                    
                    if phase == 'train' : 
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item()
                running_corr += (output.argmax(1)==y).sum().item()
            
            epoch_loss = running_loss / total
            epoch_acc = running_corr / total

            if phase == 'valid' and epoch_loss < best_val_loss : 
                print(f'On Epoch {epoch}, Best Model Saved with Valid Loss {round(epoch_loss, 6)} and Acc {round(epoch_acc, 4)*100}%')
                
                best_val_loss = epoch_loss
                best_acc = epoch_acc
                torch.save(model.state_dict(), model_path)
                early_stop_epoch = 0
        
            elif phase == 'valid' : 
                early_stop_epoch += 1

        if early_stop_epoch >= early_stop : 
            "Early Stop Occured on epoch" + str(epoch)
            break;
    time_elapsed = time.time() - start_time
    print(f'Training Complete in {time_elapsed//60}min {time_elapsed%60}sec')
    print(f'Best Validation Loss : {round(best_val_loss, 6)} with Accuracy {round(best_acc,4)*100}%')


    model.load_state_dict(torch.load(model_path))
    return model

In [6]:
fcnn = FashionCNN().to(device)
dataloader = return_dataloader(train_idx, valid_idx)
optimizer = optim.Adam(fcnn.parameters(), lr = 0.005)
loss_fn = nn.CrossEntropyLoss()
trained_fcnn = train_model(fcnn, dataloader, optimizer, loss_fn, num_epoch = 100, early_stop = 5, model_path =  'FCNN.pth')

On Epoch 0, Best Model Saved with Valid Loss 0.003605 and Acc 65.95%
On Epoch 1, Best Model Saved with Valid Loss 0.00356 and Acc 67.71000000000001%
On Epoch 2, Best Model Saved with Valid Loss 0.00351 and Acc 70.27%
On Epoch 3, Best Model Saved with Valid Loss 0.003326 and Acc 79.74%
On Epoch 4, Best Model Saved with Valid Loss 0.003133 and Acc 89.68%
On Epoch 5, Best Model Saved with Valid Loss 0.003132 and Acc 89.55%
On Epoch 6, Best Model Saved with Valid Loss 0.003132 and Acc 89.57000000000001%
On Epoch 7, Best Model Saved with Valid Loss 0.003105 and Acc 90.88000000000001%
On Epoch 8, Best Model Saved with Valid Loss 0.003104 and Acc 90.93%
On Epoch 10, Best Model Saved with Valid Loss 0.003091 and Acc 91.49000000000001%
On Epoch 12, Best Model Saved with Valid Loss 0.003088 and Acc 91.72%
On Epoch 15, Best Model Saved with Valid Loss 0.003087 and Acc 91.7%
On Epoch 20, Best Model Saved with Valid Loss 0.003081 and Acc 92.10000000000001%
On Epoch 21, Best Model Saved with Valid L

In [7]:
test_dataloder = DataLoader(dataset = fmnist_test, batch_size  = batch_size , shuffle = False)
class_correct = [0 for x in fmnist_test.classes]
total_correct = [0 for x in fmnist_test.classes]
with torch.no_grad() : 
    for images, labels in test_dataloder : 
        images = images.to(device)
        prediction_soft = trained_fcnn(images)
        prediction_hard = prediction_soft.argmax(1)

        correction = (prediction_hard.cpu() == labels).squeeze()

        for label,corr in zip(labels, correction) : 
            total_correct[label] += 1
            if corr : 
                class_correct[label] += 1
acc_per_class = [cl/to for cl, to in zip(class_correct, total_correct)]

- 옷 종류에 대해서 혼동하는 양상을 보임

In [8]:
for cls, acc in zip(fmnist_test.classes, acc_per_class) : 
    print(f'Accuracy of {cls} : {round(acc*100,2)}%')
print('')
print(f'Total Accuracy : {round(sum(class_correct) / sum(total_correct), 4)}%')

Accuracy of T-shirt/top : 86.9%
Accuracy of Trouser : 98.1%
Accuracy of Pullover : 84.4%
Accuracy of Dress : 92.0%
Accuracy of Coat : 89.8%
Accuracy of Sandal : 98.0%
Accuracy of Shirt : 75.4%
Accuracy of Sneaker : 98.9%
Accuracy of Bag : 99.2%
Accuracy of Ankle boot : 94.9%

Total Accuracy : 0.9176%
